Código web-scraping

In [1]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import re
import pandas as pd
import numpy as np

In [14]:
ementa = list()
processo = list()
numero_processo = list()
agencia_envolvida = list()
relator = list()
turma = list()
data_julgamento = list()
data_publicacao = list()
agencia_envolvida = list()

In [ ]:
agencias = ['ANATEL', 'ANEEL', 'ANCINE', 'ANAC', 'ANTAQ', 'ANTT', 'ANP', 'ANVISA', 'ANS', 'ANM']

for agencia in agencias:
    path = 'D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa'
    final_path = 'D:/HD externo yas/Faculdade/Assistencia de pesquisa'

    chrome_options = Options()
    chrome_options.add_argument("--window-size=1240,1080") # define o tamanho da janela a ser aberta
    driver = webdriver.Chrome(executable_path = os.path.join(path,'chromedriver.exe'), options = chrome_options) # o path do chromedriver
    url = 'https://scon.stj.jus.br/SCON/pesquisar.jsp'
    driver.get(url)
    
    html_source = driver.page_source
    soup = BeautifulSoup(html_source,"html.parser")

    pesquisa = driver.find_elements(by=By.XPATH, value='//*[@id="pesquisaLivre"]')
    #time.sleep(5)
    pesquisa[0].send_keys(f'"Recurso Especial" {agencia} nao embargo nao agravo') # buscando os acórdãos desejados

    pesquisa_botao = driver.find_element(by=By.XPATH, value='//*[@id="frmConsulta"]/div[3]/div[2]/div[2]/div/div/button') # salvando o botão de clique de busca
    #time.sleep(3)
    pesquisa_botao.click() # clicando na busca

    for pagina in range(20):
        
        html_source = driver.page_source
        soup = BeautifulSoup(html_source,"html.parser") # salvando a página encontrada pela busca

        aumentar_acordaos = Select (driver.find_element(by=By.XPATH, value='//*[@id="qtdDocsPagina"]')) # expandindo o número de acórdãos por página
        aumentar_acordaos.select_by_value('50')
        
        simplificaco_botao = driver.find_element(by=By.XPATH, value='//*[@id="tp_vis_lista_resumida"]/span') # esse botão simplificará a página, facilitando a busca pelo texto da ementa
        #time.sleep(3)
        simplificaco_botao.click()
    
        html_source = driver.page_source
        soup = BeautifulSoup(html_source,"html.parser") # salvando a página simplificada

        for i in range(0,len(soup.find_all('div',{'class':'row itemlistadocumentos p-2'}))): # obtendo a ementa
            ementa.append(soup.find_all('div', {'class':'clsEmentaCompleta'})[i].get_text().replace('\n','').replace('\t',''))
        
        completo_botao = driver.find_element(by=By.XPATH, value='//*[@id="tp_vis_documento_completo"]/span') # esse botão volta a página à sua forma inicial, facilitando a busca por outros elementos
        #time.sleep(3)
        completo_botao.click()

        html_source = driver.page_source
        soup = BeautifulSoup(html_source,"html.parser") # salvando a página completa

        for i in range(0, len(soup.find_all('div',{'class':'documento'}))): 
            agencia_envolvida.append(agencia) # salvando a agência que está sendo buscada
            
        for i in range(0,len(soup.find_all('div',{'class':'col-md-6 col-sm-12'})),2):
            processo.append(re.search('\d{3,}\s\/\s\w\w', soup.find_all('div',{'class':'col-md-6 col-sm-12'})[i].get_text())[0]) # salvando o elemento com detalhes de identificação do processo
            numero_processo.append(re.sub('\/','',re.sub('-', '', re.search('\d{4,}\/\d{1,}\-\d{1,}', soup.find_all('div',{'class':'col-md-6 col-sm-12'})[i].get_text())[0])))

        for i in range(0,len(soup.find_all('div',{'class':'col-md-3 col-sm-12'})),4): # esse elemento é uma lista de quatro elementos semelhantes, então o contador pulará de quatro em quatro
            relator.append(re.subn('Relator[a]?', '', re.subn('\\n','',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[0].get_text())[0])[0]) # salvando o relator
            turma.append(re.search('(?<=\\n\\nÓrgão Julgador\\n).*(?=\\n\\n)',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[i + 1].get_text())[0]) # salvando a turma
            data_julgamento.append(re.search('\d\d\/\d\d\/\d\d\d\d',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[i + 2].get_text())[0]) # salvando a data
            data_publicacao.append(re.search('\d\d\/\d\d\/\d\d\d\d',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[i + 3].get_text())[0]) # salvando a data
        
        try:
            if pagina == 0:
                #time.sleep(3)
                driver.find_elements(by=By.XPATH, value='//*[@id="bottom-navigation"]/div[2]/a[1]/span')[0].click() # clique para a próxima página quando se está na primeira página
            else:
                #time.sleep(3)
                driver.find_elements(by=By.XPATH, value='//*[@id="navegacao"]/div[2]/a[3]/span')[0].click() # clique para a próxima página quando não se está na primeira página
        except:
            break

    driver.quit()

C:\Users\silva\AppData\Local\Temp/ipykernel_13984/2066636199.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = os.path.join(path,'chromedriver.exe'), options = chrome_options) # o path do chromedriver


In [16]:
dados1 = {'numero_processo': numero_processo,
    'processo': processo,
    'ministro_relator': relator,
    'orgao_julgador': turma,
    'agencia': agencia_envolvida,
    'data_julgamento': data_julgamento,
    'data_publicacao': data_publicacao,
    'ementa': ementa
} # salvando todas as listas obtidas num dicionário

In [17]:
df1 = pd.DataFrame.from_dict(dados1, orient='index')
df1 = df1.transpose()

In [18]:
recorrente = []
recorrido = []
numero_processo_recorrente = []
numero_processo_recorrido = []

for processo in range(0,len(df1.numero_processo)):
    print(processo)
    path = 'D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa'
    final_path = 'D:/HD externo yas/Faculdade/Assistencia de pesquisa'
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1240,1080") # define o tamanho da janela a ser aberta
    driver = webdriver.Chrome(executable_path = os.path.join(path,'chromedriver.exe'), options = chrome_options) # o path do chromedriver
    url = f"https://processo.stj.jus.br/processo/pesquisa/?src=1.1.3&aplicacao=processos.ea&tipoPesquisa=tipoPesquisaGenerica&num_registro={df1['numero_processo'][processo]}"
    driver.get(url)

    html_source = driver.page_source
    soup = BeautifulSoup(html_source,"html.parser") # salvando a página encontrada pela busca
    
    for elemento in range(0,len(soup.find_all('span',{'class':'classSpanDetalhesLabel'}))):
        texto = soup.find_all('span',{'class':'classSpanDetalhesLabel'})[elemento].get_text()
        if texto == 'RECORRENTE:' or texto == 'RECORRENTE :' or texto == 'SUSCITANTE:' or texto == 'SUSCITANTE :' or texto == 'EMBARGANTE:' or texto == 'EMBARGANTE :':
            recorrente.append(re.sub('\n','',soup.find_all('span',{'class':'classSpanDetalhesTexto'})[elemento].get_text()))
            numero_processo_recorrente.append(df1['numero_processo'][processo])
        if texto == 'RECORRIDO:' or texto == 'RECORRIDO :' or texto == 'EMBARGADO:' or texto == 'EMBARGADO :' or texto == 'SUSCITADO:' or texto == 'SUSCITADO :':
            recorrido.append(re.sub('\n','',soup.find_all('span',{'class':'classSpanDetalhesTexto'})[elemento].get_text()))
            numero_processo_recorrido.append(df1['numero_processo'][processo])

    driver.quit()

0


C:\Users\silva\AppData\Local\Temp/ipykernel_13984/3629174571.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = os.path.join(path,'chromedriver.exe'), options = chrome_options) # o path do chromedriver


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [19]:
dados_recorrente = {'numero_processo': numero_processo_recorrente,
    'recorrentes': recorrente
}
df_recorrente = pd.DataFrame.from_dict(dados_recorrente)

dados_recorrido = {'numero_processo': numero_processo_recorrido,
    'recorridos': recorrido
} 
df_recorrido = pd.DataFrame.from_dict(dados_recorrido)

df2 = df_recorrente.merge(df_recorrido, on='numero_processo', how='outer')

In [20]:
df2 = df2.groupby('numero_processo', as_index=False).agg({'numero_processo' : 'first', 'recorrentes' : '; '.join, 'recorridos' : '; '.join})
df2 = df2.drop_duplicates()
df3 = df1.merge(df2, on='numero_processo', how='left')
df3 = df3[["processo","numero_processo","ministro_relator","orgao_julgador","recorrentes","recorridos","data_julgamento","data_publicacao","ementa"]]

In [21]:
df3 = df3[["processo","numero_processo","ministro_relator","orgao_julgador","recorrentes","recorridos","data_julgamento","data_publicacao","ementa"]]

In [22]:
resultado = []
for decisao in range(0, len(df3.ementa)):
    if re.search(r"(parcial(mente)? provi(do|mento)|provido(s)? em parte)", df3['ementa'][decisao], flags=re.I | re.S):
        resultado.append("provimento parcial")
    elif re.search(r"([Nn][ãa]o |[Dd]es|[Ii]m|[Nn]egar |NÃO |NEGAR )(provido|conhecido|provimento|PROVIDO|PROVIMENTO)(s)?", df3['ementa'][decisao], flags=re.I | re.S):
        resultado.append("não provimento")
    else:
        resultado.append("provimento")
df3['resultado'] = resultado

In [23]:
df3['ministro_relator'] = df3['ministro_relator'].str.replace(r'Ministro ', '')
df3['ministro_relator'] = df3['ministro_relator'].str.replace(r'Ministra ', '')
df3['ministro_relator'] = df3['ministro_relator'].str.replace(r' \W\d{4}\W', '')
df3

C:\Users\silva\AppData\Local\Temp/ipykernel_13984/1098079465.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df3['ministro_relator'] = df3['ministro_relator'].str.replace(r' \W\d{4}\W', '')


,processo,numero_processo,ministro_relator,orgao_julgador,recorrentes,recorridos,data_julgamento,data_publicacao,ementa,resultado
0,1884231 / GO,202001738005,MAURO CAMPBELL MARQUES,T2 - SEGUNDA TURMA,CELG DISTRIBUICAO S.A. - CELG D,MUNICIPIO DE MINACU,22/03/2022,28/03/2022,PROCESSUAL CIVIL. ADMINISTRATIVO. EM RECURSO E...,não provimento
1,1946665 / MA,202102021701,MAURO CAMPBELL MARQUES,T2 - SEGUNDA TURMA,EQUATORIAL MARANHAO DISTRIBUIDORA DE ENERGIA S.A,POSTO BACANGA LTDA,05/10/2021,15/10/2021,ADMINISTRATIVO. RECURSO ESPECIAL. ENERGIA ELÉT...,não provimento
2,1786167 / SP,201802985288,MAURO CAMPBELL MARQUES,T2 - SEGUNDA TURMA,COMPANHIA PAULISTA DE FORCA E LUZ; AGÊNCIA NAC...,OS MESMOS; OS MESMOS,27/04/2021,01/07/2021,ADMINISTRATIVO E PROCESSUAL CIVIL. SERVIÇOS PÚ...,não provimento
3,1815394 / PR,201901371710,MAURO CAMPBELL MARQUES,T2 - SEGUNDA TURMA,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA - ANEEL,COPEL GERACAO E TRANSMISSAO S.A,16/03/2021,07/04/2021,ADMINISTRATIVO. AÇÃO ANULATÓRIA. PROCESSO ADMI...,não provimento
4,1809719 / DF,201901073956,MAURO CAMPBELL MARQUES,T2 - SEGUNDA TURMA,TELEMONT ENGENHARIA DE TELECOMUNICACOES S/A,DISTRITO FEDERAL,23/06/2020,25/06/2020,TRIBUTÁRIO. RECURSO ESPECIAL. ENUNCIADO ADMINI...,não provimento
...,...,...,...,...,...,...,...,...,...,...
207,494828 / SP,200201665160,GILSON DIPP,T6 - SEXTA TURMA,THIAGO VALQUES MARTINS (PRESO),MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO,26/05/2004,03/09/2007,PENAL. RECURSO ESPECIAL. CLORETO DE ETILA. SUB...,não provimento
208,474508 / SP,200201289807,GILSON DIPP,T6 - SEXTA TURMA,MINISTÉRIO PÚBLICO FEDERAL,JUNISMAR DE OLIVEIRA (PRESO),06/02/2003,24/02/2003,PENAL. CLORETO DE ETILA (LANÇA-PERFUME). ABOLI...,provimento
209,434303 / PR,200200074557,GILSON DIPP,T2 - SEGUNDA TURMA,MUNICÍPIO DE CURITIBA,HONINTEG - COMPLEMENTOS ALIMENTARES LTDA,05/09/2002,30/09/2002,PROCESSUAL - COMERCIALIZAÇÃO DE PRODUTOS NATUR...,não provimento
210,299659 / RJ,200100036724,GILSON DIPP,T5 - QUINTA TURMA,MARCOS GALDINO DOS SANTOS,MINISTÉRIO PÚBLICO DO ESTADO DO PARANÁ,07/02/2002,18/03/2002,RECURSO ESPECIAL. PENAL E ADMINISTRATIVO. NORM...,provimento


In [24]:
df3.to_excel('D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa/ANEEL, ANVISA.xlsx', index=False)

Código leitor de PDF

In [25]:
# instalando o pacote direto no jupyter
# %pip install pypdf2

# importa as bibliotecas necessárias
import PyPDF2

In [26]:
path = "D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa/acordaos_pdf"
leitor_pdf = pd.DataFrame(columns = ['pagina_1', 'pagina_2', 'pagina_3', 'pagina_4', 'processo', 'numero_processo', 'ministro_relator','orgao_julgador', 'acordao', 'ementa'])

for i in os.listdir(path):
    if i.endswith(".pdf"):
       string1 = PyPDF2.PdfReader(open(path + "/" + i, 'rb')).pages[0].extract_text()
       string2 = PyPDF2.PdfReader(open(path + "/" + i, 'rb')).pages[1].extract_text()
       string3 = PyPDF2.PdfReader(open(path + "/" + i, 'rb')).pages[2].extract_text()
       string4 = PyPDF2.PdfReader(open(path + "/" + i, 'rb')).pages[3].extract_text()
       leitor_pdf = leitor_pdf.append({'pagina_1' : string1, 'pagina_2' : string2, 'pagina_3' : string3, 'pagina_4' : string4}, ignore_index = True)

leitor_pdf['acordao'] = leitor_pdf['pagina_1'] + ' ' + leitor_pdf['pagina_2'] + ' ' + leitor_pdf['pagina_3'] + ' ' + leitor_pdf['pagina_4']  
leitor_pdf = leitor_pdf.drop(columns=['pagina_1', 'pagina_2', 'pagina_3', 'pagina_4'])

In [27]:
ementa2 = []
processo2 = []
numero_processo2 = []
ministro2 = []
orgao2 = []

for acordao in range(0, len(leitor_pdf.ementa)):
    match_ementa = re.search(r"(?<=EMENTA)(.|\r|\n)*(?=ACÓRDÃO)", leitor_pdf['acordao'][acordao]).group(0).replace('\n','').replace('\t','')
    ementa2.append(match_ementa)
    match_pro = re.search(r"(?<= Nº )(.*)(?= )", leitor_pdf['acordao'][acordao]).group(0).replace('-','/')
    processo2.append(match_pro)
    match_num = re.search(r"\(([^()]+)\)", leitor_pdf['acordao'][acordao]).group(0).replace('(','').replace(')','').replace('-','').replace('/','')
    numero_processo2.append(match_num)
    match_ministro = re.search(r"(?<=MINISTRO )(.|\r|\n)*?(?=\nRECORRENTE)", leitor_pdf['acordao'][acordao]).group(0)
    ministro2.append(match_ministro)
    match_orgao = re.search(r"(?<=Ministros da )(.|\r|\n)*(?= do Superior Tribunal de Justiça)", leitor_pdf['acordao'][acordao]).group(0)
    orgao2.append(match_orgao)

leitor_pdf['ementa'] = ementa2
leitor_pdf['processo'] = processo2
leitor_pdf['numero_processo'] = numero_processo2
leitor_pdf['ministro_relator'] = ministro2
leitor_pdf['orgao_julgador'] = orgao2

In [28]:
leitor_pdf = leitor_pdf.replace('Primeira Turma','T1 - PRIMEIRA TURMA')
leitor_pdf = leitor_pdf.replace('Segunda Turma','T2 - SEGUNDA TURMA')
leitor_pdf = leitor_pdf.replace('Terceira Turma','T3 - TERCEIRA TURMA')
leitor_pdf = leitor_pdf.replace('Quarta Turma','T4 - QUARTA TURMA')
leitor_pdf = leitor_pdf.replace('Quinta Turma','T5 - QUINTA TURMA')
leitor_pdf = leitor_pdf.replace('Sexta Turma','T6 - SEXTA TURMA')

In [29]:
recorrente2 = []
recorrido2 = []
numero_processo_recorrente2 = []
numero_processo_recorrido2 = []

for processo2 in range(0,len(leitor_pdf.numero_processo)):
    print(processo2)
    path = 'D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa'
    final_path = 'D:/HD externo yas/Faculdade/Assistencia de pesquisa'
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1240,1080") # define o tamanho da janela a ser aberta
    driver = webdriver.Chrome(executable_path = os.path.join(path,'chromedriver.exe'), options = chrome_options) # o path do chromedriver
    url = f"https://processo.stj.jus.br/processo/pesquisa/?src=1.1.3&aplicacao=processos.ea&tipoPesquisa=tipoPesquisaGenerica&num_registro={leitor_pdf['numero_processo'][processo2]}"
    driver.get(url)

    html_source = driver.page_source
    soup = BeautifulSoup(html_source,"html.parser") # salvando a página encontrada pela busca
    
    for elemento2 in range(0,len(soup.find_all('span',{'class':'classSpanDetalhesLabel'}))):
        texto = soup.find_all('span',{'class':'classSpanDetalhesLabel'})[elemento2].get_text()
        if texto == 'RECORRENTE:' or texto == 'RECORRENTE :' or texto == 'SUSCITANTE:' or texto == 'SUSCITANTE :' or texto == 'EMBARGANTE:' or texto == 'EMBARGANTE :':
            recorrente2.append(re.sub('\n','',soup.find_all('span',{'class':'classSpanDetalhesTexto'})[elemento2].get_text()))
            numero_processo_recorrente2.append(leitor_pdf['numero_processo'][processo2])
        if texto == 'RECORRIDO:' or texto == 'RECORRIDO :' or texto == 'EMBARGADO:' or texto == 'EMBARGADO :' or texto == 'SUSCITADO:' or texto == 'SUSCITADO :':
            recorrido2.append(re.sub('\n','',soup.find_all('span',{'class':'classSpanDetalhesTexto'})[elemento2].get_text()))
            numero_processo_recorrido2.append(leitor_pdf['numero_processo'][processo2])

    driver.quit()

0


C:\Users\silva\AppData\Local\Temp/ipykernel_13984/1766414989.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = os.path.join(path,'chromedriver.exe'), options = chrome_options) # o path do chromedriver


1


In [30]:
dados_recorrente2 = {'numero_processo': numero_processo_recorrente2,
    'recorrentes': recorrente2
}
df_recorrente2 = pd.DataFrame.from_dict(dados_recorrente2)

dados_recorrido2 = {'numero_processo': numero_processo_recorrido2,
    'recorridos': recorrido2
} 
df_recorrido2 = pd.DataFrame.from_dict(dados_recorrido2)

In [31]:
leitor_pdf2 = df_recorrente2.merge(df_recorrido2, on='numero_processo', how='outer')

leitor_pdf2 = leitor_pdf2.groupby('numero_processo', as_index=False).agg({'numero_processo' : 'first', 'recorrentes' : '; '.join, 'recorridos' : '; '.join})

leitor_pdf2 = leitor_pdf2.drop_duplicates()

leitor_pdf3 = leitor_pdf.merge(leitor_pdf2, on='numero_processo', how='left')

In [32]:
np_list = leitor_pdf3['numero_processo'].values.tolist()
data_julgamento2 = []
data_publicacao2 = []

for processo3 in np_list:
    
    print(processo3)
    path = 'D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa'
    final_path = 'D:/HD externo yas/Faculdade/Assistencia de pesquisa'
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1240,1080") # define o tamanho da janela a ser aberta

    driver = webdriver.Chrome(executable_path = os.path.join(path,'chromedriver.exe'), options = chrome_options) # o path do chromedriver
    url = 'https://scon.stj.jus.br/SCON/pesquisar.jsp'
    driver.get(url)
    
    pesquisa = driver.find_elements(by=By.XPATH, value='//*[@id="pesquisaLivre"]')
    #time.sleep(5)
    pesquisa[0].send_keys(f'{processo3}') # buscando os acórdãos desejados

    pesquisa_botao = driver.find_element(by=By.XPATH, value='//*[@id="frmConsulta"]/div[3]/div[2]/div[2]/div/div/button') # salvando o botão de clique de busca
    #time.sleep(3)
    pesquisa_botao.click() # clicando na busca

    html_source = driver.page_source
    soup = BeautifulSoup(html_source,"html.parser") # salvando a página encontrada pela busca

    for i in range(0,len(soup.find_all('div',{'class':'col-md-3 col-sm-12'})),4): # esse elemento é uma lista de quatro elementos semelhantes, então o contador pulará de quatro em quatro
        data_julgamento2.append(re.search('\d\d\/\d\d\/\d\d\d\d',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[i + 2].get_text())[0]) # salvando a data
        data_publicacao2.append(re.search('\d\d\/\d\d\/\d\d\d\d',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[i + 3].get_text())[0]) # salvando a data

    driver.quit()

201700855717


C:\Users\silva\AppData\Local\Temp/ipykernel_13984/1190717201.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = os.path.join(path,'chromedriver.exe'), options = chrome_options) # o path do chromedriver


201901306960


In [33]:
dados_pdf = {'numero_processo': np_list, 'data_julgamento': data_julgamento2,
    'data_publicacao': data_publicacao2} # salvando todas as listas obtidas num dicionário

In [34]:
leitor_pdf_datas = pd.DataFrame.from_dict(dados_pdf, orient='index')
leitor_pdf_datas = leitor_pdf_datas.transpose()

In [35]:
leitor_pdf4 = leitor_pdf3.merge(leitor_pdf_datas, on='numero_processo', how='left')
leitor_pdf4 = leitor_pdf4.drop(columns=['acordao'])
leitor_pdf4 = leitor_pdf4[["processo","numero_processo","ministro_relator","orgao_julgador","recorrentes","recorridos","data_julgamento","data_publicacao","ementa"]]

In [36]:
resultado2 = []

for decisao in range(0, len(leitor_pdf4.ementa)):
    if re.search(r"(parcial(mente)? provi(do|mento)|provido(s)? em parte)", leitor_pdf4['ementa'][decisao], flags=re.I | re.S):
        resultado2.append("provimento parcial")
    elif re.search(r"([Nn][ãa]o |[Dd]es|[Ii]m|[Nn]egar |NÃO |NEGAR )(provido|conhecido|provimento|PROVIDO|PROVIMENTO)(s)?", leitor_pdf4['ementa'][decisao], flags=re.I | re.S):
        resultado2.append("não provimento")
    else:
        resultado2.append("provimento")
leitor_pdf4['resultado'] = resultado2
leitor_pdf4

,processo,numero_processo,ministro_relator,orgao_julgador,recorrentes,recorridos,data_julgamento,data_publicacao,ementa,resultado
0,1.669.409 / RJ,201700855717,HERMAN BENJAMIN,T2 - SEGUNDA TURMA,LEONARDO MARTINS LIMA,AGÊNCIA NACIONAL DO CINEMA,20/06/2017,30/06/2017,ADMINISTRATIVO E PROCESSUAL CIVIL. SERVIDOR PÚ...,não provimento
1,1.821.321 / SC,201901306960,HERMAN BENJAMIN\nR.P/ACÓRDÃO :MINISTRO MAURO C...,SEGUNDA TURMA,UNIÃO,SETEP CONSTRUÇÕES S/A,08/11/2022,13/12/2022,ADMINISTRATIVO. AÇÃO CIVIL PÚBLICA AJUIZADA PE...,provimento


In [45]:
df4 = pd.concat([df3, leitor_pdf4], ignore_index=True)
df4

C:\Users\silva\AppData\Local\Temp/ipykernel_4560/2712595012.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df3['ministro_relator'] = df3['ministro_relator'].str.replace(r' \W\d{4}\W', '')


,processo,numero_processo,ministro_relator,orgao_julgador,recorrentes,recorridos,data_julgamento,data_publicacao,ementa,resultado
0,1287461 / SP,201101779570,OG FERNANDES,T2 - SEGUNDA TURMA,AGENCIA NACIONAL DE AVIACAO CIVIL - ANAC,PANTANAL LINHAS AÉREAS S/A - EM RECUPERAÇÃO JU...,21/06/2022,30/06/2022,PROCESSUAL CIVIL. ADMINISTRATIVO. RECURSO ESPE...,provimento parcial
1,1611915 / RS,201600856759,OG FERNANDES,T4 - QUARTA TURMA,WEBJET LINHAS AEREAS S/A,MAURICIO BORGES ZORTEA,06/12/2018,04/02/2019,RECURSO ESPECIAL - AÇÃO CONDENATÓRIA - ACESSIB...,não provimento
2,1769070 / SP,201802491356,OG FERNANDES,T2 - SEGUNDA TURMA,FAZENDA NACIONAL; FAZENDA NACIONAL,COMERCIAL DISCOLINS LTDA; FRANCISCO ESCUDEIRO,13/11/2018,21/11/2018,PROCESSUAL CIVIL E TRIBUTÁRIO. EXECUÇÃO FISCAL...,provimento
3,1726216 / RJ,201800415171,OG FERNANDES,T2 - SEGUNDA TURMA,ALESSANDRO BARBOSA FERNANDES,AGENCIA NACIONAL DE AVIACAO CIVIL - ANAC,15/05/2018,21/11/2018,DIREITO DO CONSUMIDOR. ESPAÇO ENTRE AS POLTRON...,provimento
4,1316372 / DF,201200620887,OG FERNANDES,T2 - SEGUNDA TURMA,MINISTÉRIO PÚBLICO FEDERAL; UNIÃO,CONSTRUÇÕES E COMÉRCIO CAMARGO CORRÊA S/A E OU...,06/12/2016,19/12/2016,PROCESSUAL CIVIL. AÇÃO CAUTELAR DE PRODUÇÃO AN...,não provimento
5,1469087 / AC,201401755271,OG FERNANDES,T2 - SEGUNDA TURMA,GOL LINHAS AEREAS INTELIGENTES S.A,MINISTÉRIO PÚBLICO DO ESTADO DO ACRE,18/08/2016,17/11/2016,CONSUMIDOR. CONCESSÃO DE SERVIÇOS AÉREOS. RELA...,não provimento
6,1303014 / RS,201101853650,OG FERNANDES,T4 - QUARTA TURMA,TAM LINHAS AEREAS S/A,MINISTÉRIO PÚBLICO DO ESTADO DO RIO GRANDE DO SUL,18/12/2014,26/05/2015,RECURSO ESPECIAL. CONSUMIDOR. PROCESSUAL CIVIL...,provimento
7,1.650.546 / PE,201700181266,HERMAN BENJAMIN,T2 - SEGUNDA TURMA,MUNICÍPIO DO CABO DE SANTO AGOSTINHO; MUNICÍPI...,MUNICÍPIO DO CABO DE SANTO AGOSTINHO; AGENCIA ...,04/08/2022,19/12/2022,PROCESSUAL CIVIL. RECURSO ESPECIAL DO ENTE PÚB...,não provimento
8,1.669.409 / RJ,201700855717,HERMAN BENJAMIN,T2 - SEGUNDA TURMA,LEONARDO MARTINS LIMA,AGÊNCIA NACIONAL DO CINEMA,20/06/2017,30/06/2017,ADMINISTRATIVO E PROCESSUAL CIVIL. SERVIDOR PÚ...,não provimento


In [38]:
leitor_pdf4.to_excel('D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa/ANM, ANCINE.xlsx', index=False)

Unindo todas as agências

In [39]:
ANAC_ANTAQ_ANTT_ANP = pd.read_excel(r'D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa/ANAC, ANTAQ, ANTT, ANP.xlsx')
ANATEL = pd.read_excel(r'D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa/ANATEL.xlsx')
ANEEL_ANVISA = pd.read_excel(r'D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa/ANEEL, ANVISA.xlsx')
ANM_ANCINE = pd.read_excel(r'D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa/ANM, ANCINE.xlsx')
ANS = pd.read_excel(r'D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa/ANS.xlsx')

In [40]:
base = pd.concat([ANAC_ANTAQ_ANTT_ANP, ANATEL, ANEEL_ANVISA, ANM_ANCINE, ANS], ignore_index=True)
base

In [42]:
base.to_excel('D:/HD_externo_yas/Faculdade/Assistencia_de_pesquisa/base_stj.xlsx', index=False)